In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import os
from keras.models import *

2023-04-19 16:27:18.757931: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = load_model('/Users/parthkalathia/Desktop/CNN_SCRATCH/MODELS/3dcnn_v2-2.h5')

2023-04-19 16:27:24.826902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
def preprocess_video(video_path, input_shape):
    # Load video and get properties
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Initialize array to store frames
    all_frames = np.zeros((frame_count, input_shape[1], input_shape[2], input_shape[3]))
    frames = np.zeros((input_shape[0], input_shape[1], input_shape[2], input_shape[3]))
        
    # Read frames and resize
    for i in range(frame_count):
        ret, frame = cap.read()
        frame_resized = cv2.resize(frame, (input_shape[2], input_shape[1]))
        all_frames[i] = frame_resized
    
    # extract equal distance frames from all_frames
    j = 0
    distance_frame = frame_count//input_shape[0]
    for i in range(input_shape[0]):
        frames[i] = all_frames[j]
        j = j + distance_frame

    
    # Normalize pixel values to [0, 1]
    frames = frames / 255.0
    
    # Close video capture
    cap.release()
    
    return frames

def make_average_predictions(video_file_path, input_shape):
        
        frame = preprocess_video(video_file_path, input_shape)
        
        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(frame, axis = 0))
        print(predicted_labels_probabilities)
        return predicted_labels_probabilities

In [10]:
frame = preprocess_video('/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/T20.mp4', (32,128,128,3))

In [13]:
frame = np.expand_dims(frame, axis = 0)

In [11]:
test_files_list = os.listdir('/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL')
predictions = []
test_files_list.remove('.DS_Store')
test_files_list.remove('T49.jpg')
test_files_list.remove('M4.mp4')
test_files_list.remove('T12.mp4')
test_files_list.remove('T16.mp4')
test_files_list.remove('T47.mp4')
test_files_list.remove('T48.mp4')
test_files_list.remove('T69.mp4')
test_files_list.remove('T70.mp4')
for test_file in test_files_list:
    file = []
    input_video_file_path = f'/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/{test_file}'
#     predictions.append([test_file, make_average_predictions(input_video_file_path, num_of_frame, model)])
    print(input_video_file_path)
    file.append(test_file)
    predictions.append(file + list(make_average_predictions(input_video_file_path, (16,128,128,3))))

/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/T20.mp4
1/1 [==============================] - 0s 131ms/step
[[0.45344293 0.19770585 0.21324335 0.09719951 0.03840826]]
/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/T8.mp4
1/1 [==============================] - 0s 126ms/step
[[0.73196054 0.02953801 0.21943052 0.01503557 0.00403537]]
/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/T9.mp4
1/1 [==============================] - 0s 133ms/step
[[0.61203694 0.19534922 0.06489866 0.05510537 0.07260986]]
/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/T19.mp4
1/1 [==============================] - 0s 144ms/step
[[0.519681   0.1326015  0.17373115 0.12633406 0.04765224]]
/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/T18.mp4
1/1 [==============================] - 0s 165ms/step
[[9.2176056e-01 5.4810353e-04 7.2078004e-02 2.1384177e-03 3.4748863e-03]]
/Users/parthkalathia/Desktop/CNN_SCRATCH/TESTFINAL/T43.mp4
1/1 [==============================] - 0s 148ms/step
[[0.24559994 0.69068

In [59]:
dataframe = pd.DataFrame(data = predictions, columns=['Video', 'TALKING', 'HEADDOWN', 'YAWNING', 'MOBILE', 'NORMAL'])
dataframe.to_csv('3dcnn_v2.csv')

In [55]:
dataframe

,Video,TALKING,HEADDOWN,YAWNING,MOBILE,NORMAL
0,T20.mp4,5.123655e-01,0.093057,3.019256e-01,7.319385e-02,1.945795e-02
1,T8.mp4,3.472211e-01,0.012238,5.898154e-01,4.740449e-02,3.321179e-03
2,T9.mp4,1.667426e-01,0.001353,4.387827e-01,2.060627e-01,1.870587e-01
3,T19.mp4,3.251607e-01,0.082796,3.624272e-01,1.162207e-01,1.133950e-01
4,T18.mp4,3.405670e-02,0.000003,9.655232e-01,2.547187e-04,1.620260e-04
5,T43.mp4,1.578831e-01,0.599953,2.113768e-02,1.744995e-01,4.652632e-02
6,T42.mp4,6.141672e-01,0.037476,3.162042e-01,1.746171e-02,1.469061e-02
7,T68.mp4,6.635724e-01,0.047682,1.879724e-01,9.134772e-02,9.424890e-03
8,T41.mp4,1.854807e-01,0.000898,7.930129e-01,1.445795e-02,6.150581e-03
9,T45.mp4,1.079674e-01,0.450097,3.810112e-01,5.557533e-02,5.349048e-03
